## Importing packages

In [1]:
import numpy as np
import pandas as pd
import os
import glob
import weka.core.jvm as jvm
import weka.core.converters as converters
from sklearn import preprocessing
from sklearn.compose import make_column_selector as selector
from scipy.io import arff
from scipy.stats import t
import weka.core.jvm as jvm
from weka.attribute_selection import ASSearch
from weka.attribute_selection import ASEvaluation
from weka.attribute_selection import AttributeSelection
import weka.core.converters as converters
import numpy as np
import pandas as pd
import os
import glob
from scipy.io import arff
from sklearn.compose import make_column_selector as selector
import time

In [2]:
jvm.start(packages=True)

DEBUG:weka.core.jvm:Adding bundled jars
DEBUG:weka.core.jvm:Classpath=['C:\\Users\\Cassio\\.conda\\envs\\pww3\\Lib\\site-packages\\javabridge\\jars\\rhino-1.7R4.jar', 'C:\\Users\\Cassio\\.conda\\envs\\pww3\\Lib\\site-packages\\javabridge\\jars\\runnablequeue.jar', 'C:\\Users\\Cassio\\.conda\\envs\\pww3\\Lib\\site-packages\\javabridge\\jars\\cpython.jar', 'c:\\Users\\Cassio\\.conda\\envs\\pww3\\lib\\site-packages\\weka\\lib\\python-weka-wrapper.jar', 'c:\\Users\\Cassio\\.conda\\envs\\pww3\\lib\\site-packages\\weka\\lib\\weka.jar']
DEBUG:weka.core.jvm:MaxHeapSize=default
DEBUG:weka.core.jvm:Package support enabled


In [3]:
# # if some WEKA package is not installed:
# package_name = "chiSquaredAttributeEval"
# package_installed = False
# for p in pkg.installed_packages():
#     if p.name == package_name:
#         package_installed = True
#         print("pkg %s already installed" % package_name)
# if not package_installed:
#     pkg.install_package(package_name)
#     print("pkg %s installed, please restart" % package_name)
#     jvm.stop()
#     sys.exit(1)
#     jvm.start(packages=True)

## Importing data

In [4]:
# Moving to project directory 
os.chdir('..')

In [5]:
# checking if current directory is mlfs
os.getcwd()

'c:\\Users\\Cassio\\Desktop\\MLFS_git\\ml_fs'

In [6]:
data_set = pd.read_csv('data\\processed\\wrapper_data_set.csv')
data_set['datasetName'] = data_set['datasetName'].str.replace("-", "_")
data_set['attributeName'] = data_set['attributeName'].str.replace(" ", "_")
data_set = data_set[['baseClassifier','datasetName','attributeName','wrapperRelevance']]
data_set

,baseClassifier,datasetName,attributeName,wrapperRelevance
0,simpleLogisticRegression,iris,petallength,no
1,simpleLogisticRegression,iris,petalwidth,yes
2,simpleLogisticRegression,iris,sepallength,no
3,simpleLogisticRegression,iris,sepalwidth,no
4,simpleLogisticRegression,labor_part1,wage-increase-second-year,yes
...,...,...,...,...
488,simpleLogisticRegression,zoo,aquatic,yes
489,simpleLogisticRegression,zoo,catsize,no
490,simpleLogisticRegression,zoo,venomous,no
491,simpleLogisticRegression,zoo,predator,no


In [7]:
# datasetName
groups = data_set.iloc[:,:].datasetName

# Importing arff files

## Storing arfffiles as dataframes

In [8]:
# reading arff files and storing in dataframes

path = os.getcwd()+'\\data\\raw'
arff_files = glob.glob(os.path.join(path, "*.arff"))
arff_files = ([s for s in arff_files if 'part' not in s])

col1 =[]
col2=[]
col3=[]
col4=[]
col5 =[]
col6=[]


for f in arff_files:
      
    data = arff.loadarff(f)
    nome = f.split("\\")[-1]
    nome = nome.replace(".arff", "")
    nome = nome.replace("-", "_")
    d =  pd.DataFrame(data[0]) # auxiliar df
    categorical_columns_selector = selector(dtype_include=object)   #treating categorical columns from arff extension
    categorical_columns = categorical_columns_selector(d)
    d[categorical_columns] = d[categorical_columns].stack().str.decode('utf-8').unstack()
    d = d.replace('?',np.NaN)
    d.columns = d.columns.str.replace(' ', '_')
    
    print('Dataset:',nome)
    print('Number of instances:',len(d))
    print('Missing values:',(d.isnull().sum().sum() / (len(d)*(len(d.columns)-1)))*100,'%')    
    print('Number of numeric attributes:',len(d.columns)-1 - (len(categorical_columns)-1))    
    print('Number of categorical attributes:',len(categorical_columns)-1)
    print('Number of classes:',len(d.iloc[:,-1].unique()))
    print('')
    
    col1.append(nome)
    col2.append(len(d))
    col3.append((d.isnull().sum().sum() / (len(d)*(len(d.columns)-1)))*100)
    col4.append(len(d.columns)-1 - (len(categorical_columns)-1))
    col5.append(len(categorical_columns)-1)
    col6.append(len(d.iloc[:,-1].unique()))

    exec('dataset_{KEY} = d.copy()'.format(KEY = nome))
    
table1 = pd.DataFrame(
 {'Dataset': col1,
  'Instances': col2,
  'Missing values %': col3,
  'Numeric attributes': col4,
  'Categorical attributess': col5,
  'Classes': col6,
    }
)


Dataset: abalone_classification
Number of instances: 4177
Missing values: 0.0 %
Number of numeric attributes: 7
Number of categorical attributes: 1
Number of classes: 28

Dataset: accent_mfcc_data_1
Number of instances: 329
Missing values: 0.0 %
Number of numeric attributes: 12
Number of categorical attributes: 0
Number of classes: 6



Dataset: avila_tr
Number of instances: 10430
Missing values: 0.0 %
Number of numeric attributes: 10
Number of categorical attributes: 0
Number of classes: 12

Dataset: avila_ts
Number of instances: 10437
Missing values: 0.0 %
Number of numeric attributes: 10
Number of categorical attributes: 0
Number of classes: 12

Dataset: balance_scale
Number of instances: 625
Missing values: 0.0 %
Number of numeric attributes: 0
Number of categorical attributes: 4
Number of classes: 3

Dataset: breast_cancer
Number of instances: 286
Missing values: 0.34965034965034963 %
Number of numeric attributes: 0
Number of categorical attributes: 9
Number of classes: 2

Dataset: cmc
Number of instances: 1473
Missing values: 0.0 %
Number of numeric attributes: 2
Number of categorical attributes: 7
Number of classes: 3

Dataset: contact_lenses
Number of instances: 24
Missing values: 0.0 %
Number of numeric attributes: 0
Number of categorical attributes: 4
Number of classes: 3

Dataset: credit_g
Number of instanc

In [9]:
table1

,Dataset,Instances,Missing values %,Numeric attributes,Categorical attributess,Classes
0,abalone_classification,4177,0.000000,7,1,28
1,accent_mfcc_data_1,329,0.000000,12,0,6
2,avila_tr,10430,0.000000,10,0,12
3,avila_ts,10437,0.000000,10,0,12
4,balance_scale,625,0.000000,0,4,3
5,breast_cancer,286,0.349650,0,9,2
6,cmc,1473,0.000000,2,7,3
7,contact_lenses,24,0.000000,0,4,3
8,credit_g,1000,0.000000,7,13,2
9,data_banknote_authentication,1372,0.000000,4,0,2


In [10]:
#https://github.com/saurabhnagrecha/Pandas-to-ARFF/blob/master/pandas2arff.py
def pandas2arff(df,filename,wekaname = "pandasdata",cleanstringdata=True,cleannan=True):
    """
    converts the pandas dataframe to a weka compatible file
    df: dataframe in pandas format
    filename: the filename you want the weka compatible file to be in
    wekaname: the name you want to give to the weka dataset (this will be visible to you when you open it in Weka)
    cleanstringdata: clean up data which may have spaces and replace with "_", special characters etc which seem to annoy Weka. 
                     To suppress this, set this to False
    cleannan: replaces all nan values with "?" which is Weka's standard for missing values. 
              To suppress this, set this to False
    """
    import re
    
    def cleanstring(s):
        if s!="?":
            return re.sub('[^A-Za-z0-9]+', "_", str(s))
        else:
            return "?"
            
    dfcopy = df #all cleaning operations get done on this copy

    
    if cleannan!=False:
        dfcopy = dfcopy.fillna(-999999999) #this is so that we can swap this out for "?"
        #this makes sure that certain numerical columns with missing values don't get stuck with "object" type
 
    f = open(filename,"w")
    arffList = []
    arffList.append("@relation " + wekaname + "\n")
    #look at each column's dtype. If it's an "object", make it "nominal" under Weka for now (can be changed in source for dates.. etc)
    for i in range(df.shape[1]):
#         print(df.columns[i])
#         print('df',df.dtypes[i])        
#         print('dfcopy',dfcopy.dtypes[i])
#         if dfcopy.dtypes[i]=='O' or (df.columns[i] in ["Class","CLASS","class"]):
        if df.dtypes[i]=='O' or (df.columns[i] in ["Class","CLASS","class"]):            
#             print('Categorical column')
            if cleannan!=False:
                dfcopy.iloc[:,i] = dfcopy.iloc[:,i].replace(to_replace=-999999999, value="?")
            if cleanstringdata!=False:
                dfcopy.iloc[:,i] = dfcopy.iloc[:,i].apply(cleanstring)
            _uniqueNominalVals = [str(_i) for _i in np.unique(dfcopy.iloc[:,i])]
            _uniqueNominalVals = ",".join(_uniqueNominalVals)
            _uniqueNominalVals = _uniqueNominalVals.replace("[","")
            _uniqueNominalVals = _uniqueNominalVals.replace("]","")
            _uniqueValuesString = "{" + _uniqueNominalVals +"}" 
            arffList.append("@attribute " + df.columns[i] + _uniqueValuesString + "\n")
        else:
#             print('Numerical column')            
            arffList.append("@attribute " + df.columns[i] + " real\n") 
            #even if it is an integer, let's just deal with it as a real number for now
    arffList.append("@data\n")           
    for i in range(dfcopy.shape[0]):#instances
        _instanceString = ""
        for j in range(df.shape[1]):#features
                if dfcopy.dtypes[j]=='O':
                    _instanceString+="\"" + str(dfcopy.iloc[i,j]) + "\""
                else:
                    _instanceString+=str(dfcopy.iloc[i,j])
                if j!=dfcopy.shape[1]-1:#if it's not the last feature, add a comma
                    _instanceString+=","
        _instanceString+="\n"
        if cleannan!=False:
            _instanceString = _instanceString.replace("-999999999.0","?") #for numeric missing values
            _instanceString = _instanceString.replace("\"?\"","?") #for categorical missing values
        arffList.append(_instanceString)
    f.writelines(arffList)
    f.close()
    del dfcopy
    return True

## Splited datasets

In [11]:
os.chdir('data\\interim')

In [12]:
# criando dataframes que foram particionados a partir dos dataframes completos
col_part = list(groups.unique()[pd.Series(groups.unique()).str.contains("part")])
variaveis = data_set.iloc[:,1:3].groupby('datasetName')['attributeName'].apply(lambda x: x.tolist()).to_dict()
for nome_dataset in col_part:
    pai = 'dataset_'+nome_dataset[:-6]
    exec('dataset_{KEY} = {KEY2}.loc[:,variaveis["{KEY}"]]'.format(KEY = nome_dataset,KEY2 = pai))
    exec('dataset_{KEY} = pd.concat([dataset_{KEY}, {KEY2}.iloc[:,-1]], axis=1)'.format(KEY = nome_dataset,KEY2 = pai))
    if not os.path.isfile(f'{nome_dataset}.arff'):
    # File does not exist, create the ARFF file
        pandas2arff(f'dataset_{nome_dataset}',f'{nome_dataset}.arff',f'{nome_dataset}')
    else:
        print(f'The file {nome_dataset} already exists.')
    # exec('pandas2arff(dataset_{KEY},  "{KEY}.arff","{KEY}")'.format(KEY = nome_dataset))  #can check first if part arff files are already created

The file labor_part1 already exists.
The file labor_part2 already exists.
The file credit_g_part1 already exists.
The file credit_g_part2 already exists.
The file hypothyroid_part1 already exists.
The file hypothyroid_part2 already exists.
The file hypothyroid_part3 already exists.
The file segment_challenge_part1 already exists.
The file segment_challenge_part2 already exists.
The file soybean_part1 already exists.
The file soybean_part2 already exists.
The file soybean_part3 already exists.
The file vote_part1 already exists.
The file vote_part2 already exists.
The file ionosphere_part1 already exists.
The file ionosphere_part2 already exists.
The file ionosphere_part3 already exists.
The file sonar_all_data_part1 already exists.
The file sonar_all_data_part2 already exists.
The file sonar_all_data_part3 already exists.
The file sonar_all_data_part4 already exists.
The file sonar_all_data_part5 already exists.


In [13]:
dataset_glass = dataset_glass.rename(columns={"'K'": "K"}) # ajusting a different name in one collumn of glass dataset

In [14]:
os.chdir(os.path.dirname(os.path.dirname(os.getcwd())))

## Storing arfffiles types

In [15]:
arff_files_dict={}
# Set the path to the folder containing ARFF files
folder_path = "data\\raw"  # Replace with your folder path

# Iterate over the files in the folder
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path) and filename.endswith(".arff"):
        print("\nLoading file:", file_path)
        try:
            data = converters.load_any_file(file_path)
            
            # Perform further processing or analysis on the data
            # ...
            # Your code here
            
            # Example: Print the dataset summary
            print("Number of instances:", data.num_instances)
            print("Number of attributes:", data.num_attributes)
            print("----------------------")

            nome = filename.split("\\")[-1]
            nome = nome.replace(".arff", "")
            nome = nome.replace("-", "_")
            print(nome)
            exec('arff_files_dict["{KEY}"] = data'.format(KEY = nome))
            
            
        except Exception as e:
            print("Error loading file:", file_path)
            print("Error message:", str(e))


Loading file: data\raw\abalone_classification.arff
Number of instances: 4177
Number of attributes: 9
----------------------
abalone_classification

Loading file: data\raw\accent-mfcc-data-1.arff
Number of instances: 329
Number of attributes: 13
----------------------
accent_mfcc_data_1

Loading file: data\raw\avila-tr.arff
Number of instances: 10430
Number of attributes: 11
----------------------
avila_tr

Loading file: data\raw\avila-ts.arff
Number of instances: 10437
Number of attributes: 11
----------------------
avila_ts

Loading file: data\raw\balance-scale.arff
Number of instances: 625
Number of attributes: 5
----------------------
balance_scale

Loading file: data\raw\breast-cancer.arff
Number of instances: 286
Number of attributes: 10
----------------------
breast_cancer

Loading file: data\raw\cmc.arff
Number of instances: 1473
Number of attributes: 10
----------------------
cmc

Loading file: data\raw\contact-lenses.arff
Number of instances: 24
Number of attributes: 5
------

Number of instances: 19020
Number of attributes: 11
----------------------
magic04

Loading file: data\raw\mammographic_masses.arff
Number of instances: 961
Number of attributes: 6
----------------------
mammographic_masses

Loading file: data\raw\monks-1_train.arff
Number of instances: 124
Number of attributes: 7
----------------------
monks_1_train

Loading file: data\raw\monks-2_train.arff
Number of instances: 169
Number of attributes: 7
----------------------
monks_2_train

Loading file: data\raw\ObesityDataSet_raw_and_data_sinthetic.arff
Number of instances: 2111
Number of attributes: 17
----------------------
ObesityDataSet_raw_and_data_sinthetic

Loading file: data\raw\seeds_dataset.arff
Number of instances: 210
Number of attributes: 8
----------------------
seeds_dataset

Loading file: data\raw\segment-challenge.arff
Number of instances: 1500
Number of attributes: 20
----------------------
segment_challenge

Loading file: data\raw\sonar_all-data.arff
Number of instances: 208
Nu

In [16]:
# Set the path to the folder containing ARFF files
folder_path = "data\\interim"  # Replace with your folder path

# Iterate over the files in the folder
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path) and filename.endswith(".arff"):
        print("\nLoading file:", file_path)
        try:
            data = converters.load_any_file(file_path)
            
            # Perform further processing or analysis on the data
            # ...
            # Your code here
            
            # Example: Print the dataset summary
            print("Number of instances:", data.num_instances)
            print("Number of attributes:", data.num_attributes)
            print("----------------------")

            nome = filename.split("\\")[-1]
            nome = nome.replace(".arff", "")
            nome = nome.replace("-", "_")
            print(nome)
            exec('arff_files_dict["{KEY}"] = data'.format(KEY = nome))
            
            
        except Exception as e:
            print("Error loading file:", file_path)
            print("Error message:", str(e))


Loading file: data\interim\credit_g_part1.arff
Number of instances: 1000
Number of attributes: 11
----------------------
credit_g_part1

Loading file: data\interim\credit_g_part2.arff
Number of instances: 1000
Number of attributes: 11
----------------------
credit_g_part2

Loading file: data\interim\hypothyroid_part1.arff
Number of instances: 3772
Number of attributes: 11
----------------------
hypothyroid_part1

Loading file: data\interim\hypothyroid_part2.arff
Number of instances: 3772
Number of attributes: 11
----------------------
hypothyroid_part2

Loading file: data\interim\hypothyroid_part3.arff
Number of instances: 3772
Number of attributes: 10
----------------------
hypothyroid_part3

Loading file: data\interim\ionosphere_part1.arff
Number of instances: 351
Number of attributes: 13
----------------------
ionosphere_part1

Loading file: data\interim\ionosphere_part2.arff
Number of instances: 351
Number of attributes: 12
----------------------
ionosphere_part2

Loading file: da

Number of instances: 57
Number of attributes: 9
----------------------
labor_part2

Loading file: data\interim\segment_challenge_part1.arff
Number of instances: 1500
Number of attributes: 11
----------------------
segment_challenge_part1

Loading file: data\interim\segment_challenge_part2.arff
Number of instances: 1500
Number of attributes: 10
----------------------
segment_challenge_part2

Loading file: data\interim\sonar_all_data_part1.arff
Number of instances: 208
Number of attributes: 13
----------------------
sonar_all_data_part1

Loading file: data\interim\sonar_all_data_part2.arff
Number of instances: 208
Number of attributes: 13
----------------------
sonar_all_data_part2

Loading file: data\interim\sonar_all_data_part3.arff
Number of instances: 208
Number of attributes: 13
----------------------
sonar_all_data_part3

Loading file: data\interim\sonar_all_data_part4.arff
Number of instances: 208
Number of attributes: 13
----------------------
sonar_all_data_part4

Loading file: 

# Descriptors

In [17]:
import re
from sklearn.linear_model import LinearRegression
from scipy import stats

def descriptors(data,data_encoded):
    # perform attribute selection
    search = ASSearch(classname="weka.attributeSelection.Ranker", options=["-N", "-1"])
    evaluation = ASEvaluation(classname="weka.attributeSelection.PrincipalComponents", options=["-R", "1.00"]) #, options=["-A", "-1"]
    attsel = AttributeSelection()
    attsel.search(search)
    attsel.evaluator(evaluation)
    attsel.select_attributes(data_encoded)

    # Sample string containing the information
    sample_string = attsel.results_string
  
    

    # Regular expressions to extract the required information
    correlation_pattern = r'Correlation matrix(.*?)\n\n'

    # Extracting the relevant information
    correlation_matrix = re.search(correlation_pattern, sample_string, re.DOTALL).group(1)

    # Splitting the values and creating lists
    correlation_values = [list(map(float, line.split())) for line in correlation_matrix.strip().split('\n')[:]]

    # Create DataFrames
    correlation_df = pd.DataFrame(correlation_values)

    # Descriptor dsMatrixCorrelSD 
    dsMatrixCorrelSD = np.std(correlation_df.values,ddof=1)

#     # perform attribute selection
#     search = ASSearch(classname="weka.attributeSelection.Ranker", options=["-N", "-1"])
#     evaluation = ASEvaluation(classname="weka.attributeSelection.PrincipalComponents", options=["-R", "1.00"]) #, options=["-A", "-1"]
#     attsel = AttributeSelection()
#     attsel.search(search)
#     attsel.evaluator(evaluation)
#     attsel.select_attributes(data)
#     # Sample string containing the information
#     sample_string = attsel.results_string
        
    # Sample string containing the information
#     sample_string = attsel.results_string

    # Regular expressions to extract the required information
    proportion_pattern = r'\n\n\neigenvalue\tproportion\tcumulative\n(.*?)\n\n'

    # Extracting the relevant information
    proportion_cumulative = re.search(proportion_pattern, sample_string, re.DOTALL).group(1)

    # Splitting the values and creating lists
    a=[]
    for line in proportion_cumulative.strip().split('\n'):
        for field in line.split('\t'):
            try:
                a.append(float(field))
    #             print(a)
            except:
                pass
    #             print("erro")    
    #   proportion_cumulative_values = [list(map(float, line.split('\t'))) for line in proportion_cumulative.strip().split('\n')]
    # Reshape the list into chunks of 3
    num_columns=3
    reshaped_values = [a[i:i + num_columns] for i in range(0, len(a), num_columns)]

    # Create a DataFrame
    proportion_cumulative_df = pd.DataFrame(reshaped_values, columns=['eigenvalue', 'proportion', 'cumulative'])



    # dsEigenvaluePropIntercept
    reg_prop = LinearRegression().fit((proportion_cumulative_df.index.values +1).reshape(-1, 1),proportion_cumulative_df.proportion.values)
    dsEigenvaluePropIntercept = reg_prop.intercept_

    # dsEigenvalueCumulativeIntercept
    reg_cumulative = LinearRegression().fit((proportion_cumulative_df.index.values +1).reshape(-1, 1),proportion_cumulative_df.cumulative.values)
    dsEigenvalueCumulativeIntercept = reg_cumulative.intercept_
    
    
#     # dsEigenvaluePropIntercept
#     x = proportion_cumulative_df.index.values +1
#     print('x:',x)
#     y = proportion_cumulative_df.proportion.values
#     print('y:',y)
#     res = stats.linregress(x, y)
#     print('intercept:',res.intercept)    
    
#     # dsEigenvalueCumulativeIntercept
#     x = proportion_cumulative_df.index.values +1
#     print('x:',x)
#     y = proportion_cumulative_df.cumulative.values
#     print('y:',y)
#     res = stats.linregress(x, y)
#     print('intercept:',res.intercept)    
    
    

    # perform attribute selection
    search = ASSearch(classname="weka.attributeSelection.Ranker", options=["-N", "-1"])
    evaluation = ASEvaluation(classname="weka.attributeSelection.ChiSquaredAttributeEval")
    attsel = AttributeSelection()
    attsel.search(search)
    attsel.evaluator(evaluation)
    attsel.select_attributes(data)

    # dsChiSquaredMax
    dsChiSquaredMax = attsel.ranked_attributes[0][1]

    # Sample string containing the information
    sample_string = attsel.results_string

    # Regular expressions to extract the required information
    chisquare_pattern = r'\n\nRanked attributes:(.*?)\n\n'

    # Extracting the relevant information
    chisquare = re.search(chisquare_pattern, sample_string, re.DOTALL).group(1)
    
    # Splitting the values and creating lists
    lines = chisquare.strip().split('\n')
    aux = [line.split(None, 2) for line in lines]

    # Create a DataFrame from the list of lists
    chisquare_df = pd.DataFrame(aux, columns=['chisquare', 'rank', 'attribute'])
    chisquare_df['chisquare'] = chisquare_df['chisquare'].astype(float)
    chisquare_df['rank'] = chisquare_df['rank'].astype(int)
    chisquare_df.chisquare = chisquare_df.chisquare/max(chisquare_df.chisquare)
    
#     # Splitting the values and creating lists
#     a=[]
#     for line in chisquare.strip().split('\n'):
#         print('line:',line)
#         for field in line.split():
#             print('field:',field)
#             try:
#                 a.append(float(field))
#     #             print(a)
#             except:
#                 a.append(field)
#     #             print("erro")    
#     #   proportion_cumulative_values = [list(map(float, line.split('\t'))) for line in proportion_cumulative.strip().split('\n')]

#     # Reshape the list into chunks of 3
#     num_columns=3
#     reshaped_values = [a[i:i + num_columns] for i in range(0, len(a), num_columns)]

#     # Create a DataFrame
#     chisquare_df = pd.DataFrame(reshaped_values, columns=['chisquare', 'rank', 'attribute'])
#     print(chisquare_df)
#     chisquare_df.chisquare = chisquare_df.chisquare/max(chisquare_df.chisquare)

    # attChiSquaredNormalized
    attChiSquaredNormalized = chisquare_df.drop(['rank'], axis=1)

    # perform attribute selection
    search = ASSearch(classname="weka.attributeSelection.Ranker", options=["-N", "-1"])
    evaluation = ASEvaluation(classname="weka.attributeSelection.ClassifierAttributeEval", options=["-B", "weka.classifiers.functions.SimpleLogistic"]) #, options=["-A", "-1"]
    attsel = AttributeSelection()
    attsel.search(search)
    attsel.evaluator(evaluation)
    attsel.select_attributes(data)

    # Sample string containing the information
    sample_string = attsel.results_string

    # Regular expressions to extract the required information
    classif_log_pattern = r'\nRanked attributes:(.*?)\n\n'

    # Extracting the relevant information
    classif_log = re.search(classif_log_pattern, sample_string, re.DOTALL).group(1)
    
    # Splitting the values and creating lists
    lines = classif_log.strip().split('\n')
    aux = [line.split(None, 2) for line in lines]

    # Create a DataFrame from the list of lists
    classif_log_df = pd.DataFrame(aux, columns=['classif_log', 'rank', 'attribute'])
    classif_log_df['classif_log'] = classif_log_df['classif_log'].astype(float)
    classif_log_df['rank'] = classif_log_df['rank'].astype(int)
    classif_log_df.classif_log = classif_log_df.classif_log/max(classif_log_df.classif_log)

#     # Splitting the values and creating lists
#     a=[]
#     for line in classif_log.strip().split('\n'):
#         for field in line.split():
#             try:
#                 a.append(float(field))
#     #             print(a)
#             except:
#                 a.append(field)
#     #             print("erro")    
#     #   proportion_cumulative_values = [list(map(float, line.split('\t'))) for line in proportion_cumulative.strip().split('\n')]

#     # Reshape the list into chunks of 3
#     num_columns=3
#     reshaped_values = [a[i:i + num_columns] for i in range(0, len(a), num_columns)]

#     # Create a DataFrame
#     classif_log_df = pd.DataFrame(reshaped_values, columns=['classif_log', 'rank', 'attribute'])
#     classif_log_df.classif_log = classif_log_df.classif_log/max(classif_log_df.classif_log)

    # attClassifierLogisticNormalized
    attClassifierLogisticNormalized = classif_log_df.drop(['rank'], axis=1)

    # perform attribute selection
    search = ASSearch(classname="weka.attributeSelection.Ranker", options=["-N", "-1"])
    evaluation = ASEvaluation(classname="weka.attributeSelection.CorrelationAttributeEval") #, options=["-A", "-1"]
    attsel = AttributeSelection()
    attsel.search(search)
    attsel.evaluator(evaluation)
    attsel.select_attributes(data)

    # Sample string containing the information
    sample_string = attsel.results_string

    # Regular expressions to extract the required information
    correlation_pattern = r'\nRanked attributes:(.*?)\n\n'

    # Extracting the relevant information
    correlation = re.search(correlation_pattern, sample_string, re.DOTALL).group(1)
    
    # Splitting the values and creating lists
    lines = correlation.strip().split('\n')
    aux = [line.split(None, 2) for line in lines]

    # Create a DataFrame from the list of lists
    correlation_df = pd.DataFrame(aux, columns=['correlation', 'rank', 'attribute'])
    correlation_df['correlation'] = correlation_df['correlation'].astype(float)
    correlation_df['rank'] = correlation_df['rank'].astype(int)
    correlation_df.correlation = correlation_df.correlation/max(correlation_df.correlation)

#     # Splitting the values and creating lists
#     a=[]
#     for line in correlation.strip().split('\n'):
#         for field in line.split():
#             try:
#                 a.append(float(field))
#     #             print(a)
#             except:
#                 a.append(field)
#     #             print("erro")    
#     #   proportion_cumulative_values = [list(map(float, line.split('\t'))) for line in proportion_cumulative.strip().split('\n')]

#     # Reshape the list into chunks of 3
#     num_columns=3
#     reshaped_values = [a[i:i + num_columns] for i in range(0, len(a), num_columns)]

#     # Create a DataFrame
#     correlation_df = pd.DataFrame(reshaped_values, columns=['correlation', 'rank', 'attribute'])
#     correlation_df.correlation = correlation_df.correlation/max(correlation_df.correlation)

    # attCorrelationNormalized
    attCorrelationNormalized = correlation_df.drop(['rank'], axis=1)

    # perform attribute selection
    search = ASSearch(classname="weka.attributeSelection.Ranker", options=["-N", "-1"])
    evaluation = ASEvaluation(classname="weka.attributeSelection.ReliefFAttributeEval") #, options=["-A", "-1"]
    attsel = AttributeSelection()
    attsel.search(search)
    attsel.evaluator(evaluation)
    attsel.select_attributes(data)

    # Sample string containing the information
    sample_string = attsel.results_string

    # Regular expressions to extract the required information
    reliefF_pattern = r'\n\nRanked attributes:(.*?)\n\n'

    # Extracting the relevant information
    relief = re.search(reliefF_pattern, sample_string, re.DOTALL).group(1)
    
    # Splitting the values and creating lists
    lines = relief.strip().split('\n')
    aux = [line.split(None, 2) for line in lines]

    # Create a DataFrame from the list of lists
    relief_df = pd.DataFrame(aux, columns=['reliefF', 'rank', 'attribute'])
    relief_df['reliefF'] = relief_df['reliefF'].astype(float)
    relief_df['rank'] = relief_df['rank'].astype(int)
    relief_df.reliefF = relief_df.reliefF/max(relief_df.reliefF)

#     # Splitting the values and creating lists
#     a=[]
#     for line in relief.strip().split('\n'):
#         for field in line.split():
#             try:
#                 a.append(float(field))
#     #             print(a)
#             except:
#                 a.append(field)
#     #             print("erro")    
#     #   proportion_cumulative_values = [list(map(float, line.split('\t'))) for line in proportion_cumulative.strip().split('\n')]

#     # Reshape the list into chunks of 3
#     num_columns=3
#     reshaped_values = [a[i:i + num_columns] for i in range(0, len(a), num_columns)]

#     # Create a DataFrame
#     relief_df = pd.DataFrame(reshaped_values, columns=['reliefF', 'rank', 'attribute'])
#     relief_df.reliefF = relief_df.reliefF/max(relief_df.reliefF)

    # attReliefFNormalized
    attReliefFNormalized = relief_df.drop(['rank'], axis=1)

    # perform attribute selection
    search = ASSearch(classname="weka.attributeSelection.Ranker", options=["-N", "-1"])
    evaluation = ASEvaluation(classname="weka.attributeSelection.SymmetricalUncertAttributeEval") #, options=["-A", "-1"]
    attsel = AttributeSelection()
    attsel.search(search)
    attsel.evaluator(evaluation)
    attsel.select_attributes(data)

    # Sample string containing the information
    sample_string = attsel.results_string

    # Regular expressions to extract the required information
    sym_uncert_pattern = r'\n\nRanked attributes:(.*?)\n\n'

    # Extracting the relevant information
    sym_uncert = re.search(sym_uncert_pattern, sample_string, re.DOTALL).group(1)
    
    # Splitting the values and creating lists
    lines = sym_uncert.strip().split('\n')
    aux = [line.split(None, 2) for line in lines]

    # Create a DataFrame from the list of lists
    sym_uncert_df = pd.DataFrame(aux, columns=['sym_uncert', 'rank', 'attribute'])
    sym_uncert_df['sym_uncert'] = sym_uncert_df['sym_uncert'].astype(float)
    sym_uncert_df['rank'] = sym_uncert_df['rank'].astype(int)
    sym_uncert_df.sym_uncert = sym_uncert_df.sym_uncert/max(sym_uncert_df.sym_uncert)

#     # Splitting the values and creating lists
#     a=[]
#     for line in sym_uncert.strip().split('\n'):
#         for field in line.split():
#             try:
#                 a.append(float(field))
#         #             print(a)
#             except:
#                 a.append(field)
#     #             print("erro")    
#     #   proportion_cumulative_values = [list(map(float, line.split('\t'))) for line in proportion_cumulative.strip().split('\n')]

#     # Reshape the list into chunks of 3
#     num_columns=3
#     reshaped_values = [a[i:i + num_columns] for i in range(0, len(a), num_columns)]

#     # Create a DataFrame
#     sym_uncert_df = pd.DataFrame(reshaped_values, columns=['sym_uncert', 'rank', 'attribute'])
#     sym_uncert_df.sym_uncert = sym_uncert_df.sym_uncert/max(sym_uncert_df.sym_uncert)

    # attSymmetricalUncertNormalized
    attSymmetricalUncertNormalized = sym_uncert_df.drop(['rank'], axis=1)
    
    
    
    #dsLOGnInstances
    dsLOGnInstances = np.log10(data.num_instances)    
    
    #dsLnNumClasses
    dsLnNumClasses = np.log(data.attribute(data.class_index).num_values)
    
#     final_df = attChiSquaredNormalized.copy()
    

    final_df = pd.merge(attChiSquaredNormalized,attClassifierLogisticNormalized,on='attribute')
    final_df = pd.merge(final_df,attCorrelationNormalized,on='attribute')
    final_df = pd.merge(final_df,attReliefFNormalized,on='attribute')
    final_df = pd.merge(final_df,attSymmetricalUncertNormalized,on='attribute') 
    
    final_df = final_df.rename(columns={"chisquare": "attChiSquaredNormalized", "classif_log": "attClassifierLogisticNormalized","correlation": "attCorrelationNormalized", "reliefF": "attReliefFNormalized","sym_uncert": "attSymmetricalUncertNormalized"})
    
    final_df['dsMatrixCorrelSD'] = dsMatrixCorrelSD
    final_df['dsEigenvaluePropIntercept'] = dsEigenvaluePropIntercept    
    final_df['dsEigenvalueCumulativeIntercept'] = dsEigenvalueCumulativeIntercept
    final_df['dsChiSquaredMax'] = dsChiSquaredMax
    final_df['dsLOGnInstances'] = dsLOGnInstances
    final_df['dsLnNumClasses'] = dsLnNumClasses
    
    
    
    final_df= final_df[['dsMatrixCorrelSD','dsEigenvaluePropIntercept','dsEigenvalueCumulativeIntercept','dsChiSquaredMax','attChiSquaredNormalized','attClassifierLogisticNormalized','attCorrelationNormalized','attReliefFNormalized','attSymmetricalUncertNormalized','dsLOGnInstances','dsLnNumClasses','attribute']]
    
    
#     final_df.merge(attClassifierLogisticNormalized, on='attribute', how='left')
#     final_df.merge(attCorrelationNormalized, on='attribute', how='left')
#     final_df.merge(attReliefFNormalized, on='attribute', how='left')    
#     final_df.merge(attSymmetricalUncertNormalized, on='attribute', how='left')      
    
#     return [dsMatrixCorrelSD,dsEigenvaluePropIntercept,dsEigenvalueCumulativeIntercept,dsChiSquaredMax,attChiSquaredNormalized,attClassifierLogisticNormalized,attCorrelationNormalized,attReliefFNormalized,attSymmetricalUncertNormalized,dsLOGnInstances,dsLnNumClasses,final_df]    
    return final_df

In [18]:
from weka.core.dataset import Instances
descriptors_df = pd.DataFrame()
datasets_descriptors =list(groups.unique())
for i in datasets_descriptors:
    exec('dataset_arff = arff_files_dict["{KEY}"]'.format(KEY = i))    
    dataset0 = Instances.copy_instances(dataset_arff)  # Copy instances from dataset
    dataset0.class_is_last()   # set class attribute
    # load a dataset
    dataset1 = Instances.copy_instances(dataset_arff)  # Copy instances from dataset
    aux = descriptors(dataset0,dataset1)
    aux['datasetName'] = i
    descriptors_df = pd.concat([descriptors_df, aux], ignore_index=True)   
    

In [19]:
descriptors_df['attribute'] = descriptors_df['attribute'].str.replace(" ", "_")
descriptors_df['datasetName'] = descriptors_df['datasetName'].str.replace("-", "_")
descriptors_df = descriptors_df.rename(columns={'attribute': 'attributeName'})

In [20]:
descriptors_df

,dsMatrixCorrelSD,dsEigenvaluePropIntercept,dsEigenvalueCumulativeIntercept,dsChiSquaredMax,attChiSquaredNormalized,attClassifierLogisticNormalized,attCorrelationNormalized,attReliefFNormalized,attSymmetricalUncertNormalized,dsLOGnInstances,dsLnNumClasses,attributeName,datasetName
0,0.666645,0.493463,0.719181,268.419048,1.000000,0.988836,1.000000,0.954787,0.925373,2.176091,1.098612,petallength,iris
1,0.666645,0.493463,0.719181,268.419048,0.994371,1.000000,0.962602,1.000000,1.000000,2.176091,1.098612,petalwidth,iris
2,0.666645,0.493463,0.719181,268.419048,0.470954,0.657097,0.777236,0.372340,0.469575,2.176091,1.098612,sepallength,iris
3,0.666645,0.493463,0.719181,268.419048,0.263171,0.357257,0.645528,0.324468,0.275545,2.176091,1.098612,sepalwidth,iris
4,0.406496,0.216496,0.398940,15.064286,1.000000,1.000000,0.878689,0.956530,1.000000,1.755875,0.693147,wage-increase-second-year,labor_part1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,0.381562,0.156774,0.517723,208.165859,0.222956,0.350015,0.463506,0.363265,0.383380,2.004321,1.945910,aquatic,zoo
489,0.381562,0.156774,0.517723,208.165859,0.178550,0.249975,0.555631,0.366118,0.299295,2.004321,1.945910,catsize,zoo
490,0.381562,0.156774,0.517723,208.165859,0.091768,0.000000,0.297375,0.046668,0.156368,2.004321,1.945910,venomous,zoo
491,0.381562,0.156774,0.517723,208.165859,0.058555,0.000000,0.154953,0.105274,0.090641,2.004321,1.945910,predator,zoo


In [21]:
descriptors_df

,dsMatrixCorrelSD,dsEigenvaluePropIntercept,dsEigenvalueCumulativeIntercept,dsChiSquaredMax,attChiSquaredNormalized,attClassifierLogisticNormalized,attCorrelationNormalized,attReliefFNormalized,attSymmetricalUncertNormalized,dsLOGnInstances,dsLnNumClasses,attributeName,datasetName
0,0.666645,0.493463,0.719181,268.419048,1.000000,0.988836,1.000000,0.954787,0.925373,2.176091,1.098612,petallength,iris
1,0.666645,0.493463,0.719181,268.419048,0.994371,1.000000,0.962602,1.000000,1.000000,2.176091,1.098612,petalwidth,iris
2,0.666645,0.493463,0.719181,268.419048,0.470954,0.657097,0.777236,0.372340,0.469575,2.176091,1.098612,sepallength,iris
3,0.666645,0.493463,0.719181,268.419048,0.263171,0.357257,0.645528,0.324468,0.275545,2.176091,1.098612,sepalwidth,iris
4,0.406496,0.216496,0.398940,15.064286,1.000000,1.000000,0.878689,0.956530,1.000000,1.755875,0.693147,wage-increase-second-year,labor_part1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,0.381562,0.156774,0.517723,208.165859,0.222956,0.350015,0.463506,0.363265,0.383380,2.004321,1.945910,aquatic,zoo
489,0.381562,0.156774,0.517723,208.165859,0.178550,0.249975,0.555631,0.366118,0.299295,2.004321,1.945910,catsize,zoo
490,0.381562,0.156774,0.517723,208.165859,0.091768,0.000000,0.297375,0.046668,0.156368,2.004321,1.945910,venomous,zoo
491,0.381562,0.156774,0.517723,208.165859,0.058555,0.000000,0.154953,0.105274,0.090641,2.004321,1.945910,predator,zoo


In [22]:
# Merging dataframes
merged_df = pd.merge(data_set,descriptors_df, on=['datasetName', 'attributeName'],how='inner')

# moving wrapper_relevance_column to last position
wrapper_relevance_column = merged_df.pop('wrapperRelevance')
merged_df = merged_df.assign(wrapperRelevance=wrapper_relevance_column)
merged_df

,baseClassifier,datasetName,attributeName,dsMatrixCorrelSD,dsEigenvaluePropIntercept,dsEigenvalueCumulativeIntercept,dsChiSquaredMax,attChiSquaredNormalized,attClassifierLogisticNormalized,attCorrelationNormalized,attReliefFNormalized,attSymmetricalUncertNormalized,dsLOGnInstances,dsLnNumClasses,wrapperRelevance
0,simpleLogisticRegression,iris,petallength,0.666645,0.493463,0.719181,268.419048,1.000000,0.988836,1.000000,0.954787,0.925373,2.176091,1.098612,no
1,simpleLogisticRegression,iris,petalwidth,0.666645,0.493463,0.719181,268.419048,0.994371,1.000000,0.962602,1.000000,1.000000,2.176091,1.098612,yes
2,simpleLogisticRegression,iris,sepallength,0.666645,0.493463,0.719181,268.419048,0.470954,0.657097,0.777236,0.372340,0.469575,2.176091,1.098612,no
3,simpleLogisticRegression,iris,sepalwidth,0.666645,0.493463,0.719181,268.419048,0.263171,0.357257,0.645528,0.324468,0.275545,2.176091,1.098612,no
4,simpleLogisticRegression,labor_part1,wage-increase-second-year,0.406496,0.216496,0.398940,15.064286,1.000000,1.000000,0.878689,0.956530,1.000000,1.755875,0.693147,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,simpleLogisticRegression,zoo,aquatic,0.381562,0.156774,0.517723,208.165859,0.222956,0.350015,0.463506,0.363265,0.383380,2.004321,1.945910,yes
489,simpleLogisticRegression,zoo,catsize,0.381562,0.156774,0.517723,208.165859,0.178550,0.249975,0.555631,0.366118,0.299295,2.004321,1.945910,no
490,simpleLogisticRegression,zoo,venomous,0.381562,0.156774,0.517723,208.165859,0.091768,0.000000,0.297375,0.046668,0.156368,2.004321,1.945910,no
491,simpleLogisticRegression,zoo,predator,0.381562,0.156774,0.517723,208.165859,0.058555,0.000000,0.154953,0.105274,0.090641,2.004321,1.945910,no


In [23]:
# Export to a text file
merged_df.to_csv('merged.txt', sep='\t', index=False)

In [24]:
os.chdir('data\\processed')

In [25]:
#save DataFrame to pickle file
merged_df.to_pickle("data_set.pkl")

In [26]:
for i in groups.unique():
    exec('dataset_{KEY}.to_pickle("dataset_{KEY}.pkl")'.format(KEY = i))

In [27]:
os.chdir(os.path.dirname(os.path.dirname(os.getcwd())))

In [28]:
#save DataFrame to pickle file
os.chdir('reports')
table1.to_pickle("table1.pkl")

### Descriptors Diabets Dataset

In [29]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
cdc_diabetes_health_indicators = fetch_ucirepo(id=891) 
  
# data (as pandas dataframes) 
X = cdc_diabetes_health_indicators.data.features 
y = cdc_diabetes_health_indicators.data.targets 
  
# # metadata 
# print(cdc_diabetes_health_indicators.metadata) 
  
# # variable information 
# print(cdc_diabetes_health_indicators.variables) 

In [30]:
big_data_set = pd.DataFrame(None)
# big_data_set = pd.concat([X.iloc[0:2000,:],y.iloc[0:2000,:]], axis=1)
big_data_set = pd.concat([X,y], axis=1)
big_data_set

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,...,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,Diabetes_binary
0,1,1,1,40,1,0,0,0,0,1,...,0,5,18,15,1,0,9,4,3,0
1,0,0,0,25,1,0,0,1,0,0,...,1,3,0,0,0,0,7,6,1,0
2,1,1,1,28,0,0,0,0,1,0,...,1,5,30,30,1,0,9,4,8,0
3,1,0,1,27,0,0,0,1,1,1,...,0,2,0,0,0,0,11,3,6,0
4,1,1,1,24,0,0,0,1,1,1,...,0,2,3,0,0,0,11,5,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253675,1,1,1,45,0,0,0,0,1,1,...,0,3,0,5,0,1,5,6,7,0
253676,1,1,1,18,0,0,0,0,0,0,...,0,4,0,0,1,0,11,2,4,1
253677,0,0,1,28,0,0,0,1,1,0,...,0,1,0,0,0,0,2,5,2,0
253678,1,0,1,23,0,0,0,0,1,1,...,0,3,0,0,0,1,7,5,1,0


In [31]:
# big_data_set = pd.read_csv("data\external\bigdataset.csv",sep=';',decimal='.')

In [32]:
# defining categorical columns
big_data_set['HighBP'] = big_data_set['HighBP'].astype('O')
big_data_set['HighChol'] = big_data_set['HighChol'].astype('O')
big_data_set['CholCheck'] = big_data_set['CholCheck'].astype('O')
big_data_set['Smoker'] = big_data_set['Smoker'].astype('O')
big_data_set['Stroke'] = big_data_set['Stroke'].astype('O')
big_data_set['HeartDiseaseorAttack'] = big_data_set['HeartDiseaseorAttack'].astype('O')
big_data_set['PhysActivity'] = big_data_set['PhysActivity'].astype('O')
big_data_set['Fruits'] = big_data_set['Fruits'].astype('O')
big_data_set['Veggies'] = big_data_set['Veggies'].astype('O')
big_data_set['HvyAlcoholConsump'] = big_data_set['HvyAlcoholConsump'].astype('O')
big_data_set['AnyHealthcare'] = big_data_set['AnyHealthcare'].astype('O')
big_data_set['NoDocbcCost'] = big_data_set['NoDocbcCost'].astype('O')
big_data_set['GenHlth'] = big_data_set['GenHlth'].astype('O')
big_data_set['DiffWalk'] = big_data_set['DiffWalk'].astype('O')
big_data_set['Sex'] = big_data_set['Sex'].astype('O')
big_data_set['Age'] = big_data_set['Age'].astype('O')
big_data_set['Education'] = big_data_set['Education'].astype('O')
big_data_set['Income'] = big_data_set['Income'].astype('O')
big_data_set['Diabetes_binary'] = big_data_set['Diabetes_binary'].astype('O')


In [33]:
os.chdir('..')
os.chdir('data\\processed')

In [34]:
# creating arff file
if not os.path.isfile('big_data_set.arff'):
# File does not exist, create the ARFF file
    start_time = time.time()
    pandas2arff(big_data_set,  "big_data_set.arff","big_data_set")
    end_time = time.time()
    time_spent = end_time - start_time
    print("Time spent: ", time_spent)
else:
    print('The file big_data_set.arff already exists.')
    


The file big_data_set.arff already exists.


In [35]:
# load a dataset
data = converters.load_any_file("big_data_set.arff")
data.class_is_last()

data2 = converters.load_any_file("big_data_set.arff")

In [36]:
start_time = time.time()

descriptors_df_bigdata = descriptors(data,data2)

end_time = time.time()
time_spent = end_time - start_time
print("Time spent: ", time_spent)

Time spent:  28248.427024126053


In [37]:
descriptors_df_bigdata

,dsMatrixCorrelSD,dsEigenvaluePropIntercept,dsEigenvalueCumulativeIntercept,dsChiSquaredMax,attChiSquaredNormalized,attClassifierLogisticNormalized,attCorrelationNormalized,attReliefFNormalized,attSymmetricalUncertNormalized,dsLOGnInstances,dsLnNumClasses,attribute
0,0.163404,0.03588,0.143081,22728.069055,1.000000,NaN,0.494489,0.469193,0.740058,5.404286,0.693147,GenHlth
1,0.163404,0.03588,0.143081,22728.069055,0.772788,NaN,1.000000,0.155079,1.000000,5.404286,0.693147,HighBP
2,0.163404,0.03588,0.143081,22728.069055,0.642423,NaN,0.824021,0.046219,0.272842,5.404286,0.693147,BMI
3,0.163404,0.03588,0.143081,22728.069055,0.532118,NaN,0.829723,0.013901,0.722517,5.404286,0.693147,DiffWalk
4,0.163404,0.03588,0.143081,22728.069055,0.447695,NaN,0.761307,0.124741,0.569867,5.404286,0.693147,HighChol
5,0.163404,0.03588,0.143081,22728.069055,0.386969,NaN,0.172938,1.000000,0.220080,5.404286,0.693147,Age
6,0.163404,0.03588,0.143081,22728.069055,0.352128,NaN,0.651083,0.031842,0.263792,5.404286,0.693147,PhysHlth
7,0.163404,0.03588,0.143081,22728.069055,0.350796,NaN,0.673888,0.007514,0.542532,5.404286,0.693147,HeartDiseaseorAttack
8,0.163404,0.03588,0.143081,22728.069055,0.308153,NaN,0.258837,0.327522,0.187762,5.404286,0.693147,Income
9,0.163404,0.03588,0.143081,22728.069055,0.177187,NaN,0.254276,0.213261,0.141861,5.404286,0.693147,Education


In [38]:
#save DataFrame to pickle file
descriptors_df_bigdata.to_pickle("descriptors_df_bigdata.pkl")

### Descriptors of Financial Dataset

In [39]:
os.chdir('..')
os.chdir('..')
os.chdir('data\\external')

In [40]:
# load a dataset
data = converters.load_any_file("big_data_set2.arff")
data.class_is_last()

data2 = converters.load_any_file("big_data_set2.arff")

In [41]:
start_time = time.time()

descriptors_df_bigdata2 = descriptors(data,data2)

end_time = time.time()
time_spent = end_time - start_time
print("Time spent: ", time_spent)

Time spent:  184.0481402873993


In [42]:
descriptors_df_bigdata2

,dsMatrixCorrelSD,dsEigenvaluePropIntercept,dsEigenvalueCumulativeIntercept,dsChiSquaredMax,attChiSquaredNormalized,attClassifierLogisticNormalized,attCorrelationNormalized,attReliefFNormalized,attSymmetricalUncertNormalized,dsLOGnInstances,dsLnNumClasses,attribute
0,0.288967,0.05152,0.649535,1013.224087,1.000000,1.000000,0.374215,0.003931,1.000000,3.771587,0.693147,Attr35
1,0.288967,0.05152,0.649535,1013.224087,0.963439,-0.277778,0.637058,0.027382,0.826957,3.771587,0.693147,Attr39
2,0.288967,0.05152,0.649535,1013.224087,0.937163,-0.222222,0.354931,0.003575,0.615657,3.771587,0.693147,Attr22
3,0.288967,0.05152,0.649535,1013.224087,0.867532,0.000000,0.146202,0.006732,0.886571,3.771587,0.693147,Attr42
4,0.288967,0.05152,0.649535,1013.224087,0.844859,-0.055556,0.046529,0.000606,0.543031,3.771587,0.693147,Attr13
...,...,...,...,...,...,...,...,...,...,...,...,...
59,0.288967,0.05152,0.649535,1013.224087,0.066328,-0.055556,0.369112,0.039090,0.065130,3.771587,0.693147,Attr20
60,0.288967,0.05152,0.649535,1013.224087,0.053758,0.000000,0.029232,0.008258,0.085213,3.771587,0.693147,Attr47
61,0.288967,0.05152,0.649535,1013.224087,0.000000,0.000000,0.020472,0.000162,0.000000,3.771587,0.693147,Attr60
62,0.288967,0.05152,0.649535,1013.224087,0.000000,0.000000,0.062513,1.000000,0.000000,3.771587,0.693147,Attr37


In [43]:
os.chdir('..')
os.chdir('processed')

In [44]:
#save DataFrame to pickle file
descriptors_df_bigdata2.to_pickle("descriptors_df_bigdata2.pkl")